In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [15]:
"""impot data select features"""
data = pd.read_csv("application_training_data_import.csv")
features = data.columns
s=data.isna().sum()
null = pd.read_csv('Null_Entries.csv')
d = null[null['flag']==0]
columns = d['colname']
new_data = data[columns]



home_loans = new_data[new_data.isnull().any(axis=1) == False]
home_loans = home_loans.reset_index()
home_loans = home_loans.drop(columns = 'index')

#print(home_loans.shape)


cols_to_remove = ['SK_ID_CURR',
'FLAG_DOCUMENT_2',
'FLAG_DOCUMENT_3',
'FLAG_DOCUMENT_4',
'FLAG_DOCUMENT_5',
'FLAG_DOCUMENT_6',  
'FLAG_DOCUMENT_7',
'FLAG_DOCUMENT_8',
'FLAG_DOCUMENT_9',
'FLAG_DOCUMENT_10',
'FLAG_DOCUMENT_11',
'FLAG_DOCUMENT_12',
'FLAG_DOCUMENT_13',
'FLAG_DOCUMENT_14',
'FLAG_DOCUMENT_15',
'FLAG_DOCUMENT_16',
'FLAG_DOCUMENT_17',
'FLAG_DOCUMENT_18',                  
'FLAG_DOCUMENT_19',
'FLAG_DOCUMENT_20',
'FLAG_DOCUMENT_21', 
'AMT_REQ_CREDIT_BUREAU_HOUR',
'AMT_REQ_CREDIT_BUREAU_DAY',
'AMT_REQ_CREDIT_BUREAU_WEEK',
'FLAG_CONT_MOBILE']




cols_to_normalize = ['EXT_SOURCE_3',
'EXT_SOURCE_2',
'AMT_GOODS_PRICE',
'AMT_ANNUITY',
'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL',
'AMT_CREDIT',
'REGION_POPULATION_RELATIVE',
'DAYS_BIRTH',
'DAYS_EMPLOYED',
'DAYS_REGISTRATION',
'DAYS_ID_PUBLISH']


"""drop columns and add normalized columns"""
home_loans_df = home_loans.drop(cols_to_remove, axis=1)
## substitute the normalized columns
df_to_normalize = home_loans_df[cols_to_normalize]
norm_values = df_to_normalize.values

"""normalize values"""
scaler = StandardScaler()
scaler.fit(norm_values)
norm_values= scaler.transform(norm_values)
norm_values

"""re-impute into data"""
home_loans_df[cols_to_normalize] = norm_values
home_loans_df.head()

,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,AMT_GOODS_PRICE,...,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY
0,-1.905793,0.0,0.0,1.0,2.0,2.0,2.0,2.0,-1.351241,-0.527764,...,2,2,WEDNESDAY,10,0,0,0,0,0,0
1,1.123108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.197991,-1.109049,...,2,2,MONDAY,9,0,0,0,0,0,0
2,0.567098,0.0,1.0,1.0,0.0,0.0,0.0,0.0,-0.868560,-0.249232,...,2,2,WEDNESDAY,16,0,0,0,0,0,0
3,-0.095792,1.0,1.0,2.0,1.0,0.0,1.0,0.0,1.086863,2.281779,...,2,2,SUNDAY,16,0,0,0,0,0,0
4,0.153597,0.0,0.0,0.0,2.0,0.0,2.0,0.0,1.035459,2.645082,...,3,3,MONDAY,16,0,0,0,0,1,1


In [118]:
# print(len(home_loans_df[home_loans_df['TARGET']==1]))
# print(len(home_loans_df[home_loans_df['TARGET']==0]))

In [117]:
# print(home_loans['TARGET'].value_counts()[0]/len(home_loans), home_loans['TARGET'].value_counts()[1]/len(home_loans))
# print(home_loans.shape)
# home_loans = home_loans.drop_duplicates()
# print(home_loans.shape)

In [412]:
"""Set the indices"""
n0 = home_loans_df[home_loans_df['TARGET']==0].shape[0]
#test set percentage size 
test_per = 0.1
n1 = home_loans_df[home_loans_df['TARGET']==1].shape[0]
extra = 0

train1_index = 12000
train0_index = int(train1)

test1_index = int(n1 - train1)
test0_index = int(test1/per1) + 0


"""Take all TARGET Home Loans"""
home_loans = shuffle(home_loans_df,random_state=0)
home_loans0 = home_loans[home_loans['TARGET']==0].iloc[:101000, :]
home_loans1 = home_loans[home_loans['TARGET']==1].iloc[:, :]

# home_loans_test0 = home_loans[home_loans['TARGET']==0].iloc[10000:101000, :]
# home_loans_test1 = home_loans[home_loans['TARGET']==1].iloc[:, :]

reduced_loans = pd.concat([home_loans0, home_loans1], axis=0)
reduced_loans = shuffle(reduced_loans,random_state=0)


reduced_loans = pd.get_dummies(reduced_loans)


##########
################
########---Split data into Test-Train Split
####################
########################
"""split dataset into training set and cross validation"""
"""training data will have 50/50 0/1 class split"""
train_df0 = reduced_loans[reduced_loans['TARGET']==0].iloc[:train0_index, :]
train_df1 = reduced_loans[reduced_loans['TARGET']==1].iloc[:train1_index, :]

train = pd.concat([train_df0, train_df1])
train = shuffle(train,random_state=43)


"""Test dataframe will have 90/10 0/1 class split"""
test_df0 = reduced_loans[reduced_loans['TARGET']==0].iloc[test0_index:, :]
test_df1 = reduced_loans[reduced_loans['TARGET']==1].iloc[test1_index:, :]

test = pd.concat([test_df0, test_df1])
test = shuffle(test,random_state=57)

del reduced_loans, home_loans, home_loans0, home_loans1
######
#########
#######-----split up the y variables
##########


"""separate out the target variable"""
train_target = train['TARGET']
train = train.drop(columns = 'TARGET')
train = pd.get_dummies(train) #One Hot Encoding
# train = train.drop(columns='NAME_INCOME_TYPE_Unemployed')
# train = train.drop(columns='GENDER_CODE_XNA')

test_target = test['TARGET']
test = test.drop(columns = 'TARGET')
test = pd.get_dummies(test) #One Hot Encoding



"""break up the data"""
X_train, y_train = np.matrix(train), np.array(train_target)
X_test, y_test = np.matrix(test), np.array(test_target)

#######
#######---Create cross validation sets-------######
########
#########

traincv1 = X_train[:int(np.floor((len(train_target)/3))), :]
target1 = y_train[:int(np.floor((len(train_target)/3)))]

traincv2 = X_train[int(np.floor((len(train_target)/3))):int(np.floor((len(train_target)*2/3))), :]
target2 = y_train[int(np.floor((len(train_target)/3))):int(np.floor((len(train_target)*2/3)))]


traincv3 = X_train[int(np.floor((len(train_target)*2/3))):, :]
target3 = y_train[int(np.floor((len(train_target)*2/3))):]


"""add test dataset"""
# test = X[int(np.floor((len(reduced_loans)*3/4))):, :]
# target_test = y[int(np.floor((len(reduced_loans)*3/4))):]


"""Trainig """
print('Training data shaped {}'.format(X_train.shape))
print(traincv1.shape)
print(traincv2.shape)
print(traincv3.shape)
print('Testing data shaped {}'.format(X_test.shape))


"""make pairs of datasets"""
CV12 = np.concatenate([traincv1, traincv2])
y12 = np.concatenate([target1, target2])

CV13 = np.concatenate([traincv1, traincv3])
y13 = np.concatenate([target1, target3])

CV23 = np.concatenate([traincv2, traincv3])
y23 = np.concatenate([target2, target3])

print('cross validation shapes')
print(CV12.shape)
print(CV13.shape)
print(CV23.shape)

Training data shaped (22000, 67)
(7333, 67)
(7333, 67)
(7334, 67)
Testing data shaped (20290, 67)
cross validation shapes
(14666, 67)
(14667, 67)
(14667, 67)


### support vector machine

### SVM Train on 1,2

In [131]:
support_vector_machine12 = SVC(kernel='rbf', gamma=0.1, C=1., random_state=None)
support_vector_machine12.fit(CV12, y12)
print('finished training on 1,2')

#y_scores = clf.decision_function(X_test)
y_scores_train = support_vector_machine12.predict(CV12)
y_scores_test = support_vector_machine12.predict(traincv3)

confusion_train = confusion_matrix(y12, y_scores_train)
confusion_test = confusion_matrix(target3, y_scores_test)


print('Trained on 1,2')
print('Train Confusion Matrix with accuracy {}'.format((confusion_train[0][0]+confusion_train[1][1])/len(y12)))
print('Train Recall: {}'.format((confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]))))
print(confusion_train)
print('')
print('Test Confusion Matrix with accuracy {}'.format((confusion_test[0][0]+confusion_test[1][1])/len(target3)))
print('Test Recall: {}'.format((confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]))))
print(confusion_test)
#precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

finished training on 1,2
Trained on 1,2
Train Confusion Matrix with accuracy 0.9218600845492977
Train Recall: 0.9475458973398276
[[5933  726]
 [ 420 7587]]

Test Confusion Matrix with accuracy 0.6588491955276793
Test Recall: 0.7422990232907588
[[1868 1473]
 [1029 2964]]


### SVM Train on 1,3

In [132]:
support_vector_machine13 = SVC(kernel='rbf', gamma=0.1, C=1., random_state=None)
support_vector_machine13.fit(CV13, y13)
print('finished training on 1,3')

#y_scores = clf.decision_function(X_test)
y_scores_train = support_vector_machine13.predict(CV13) 
y_scores_test = support_vector_machine13.predict(traincv2)

confusion_train = confusion_matrix(y13, y_scores_train)
confusion_test = confusion_matrix(target2, y_scores_test)


print('Trained on 1,3')
print('Train Confusion Matrix with accuracy {}'.format((confusion_train[0][0]+confusion_train[1][1])/len(y13)))
print('Train Recall: {}'.format((confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]))))
print(confusion_train)
print('')
print('Test Confusion Matrix with accuracy {}'.format((confusion_test[0][0]+confusion_test[1][1])/len(target2)))
print('Test Recall: {}'.format((confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]))))
print(confusion_test)
#precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

finished training on 1,3
Trained on 1,3
Train Confusion Matrix with accuracy 0.9224108542987659
Train Recall: 0.9387984981226534
[[6028  649]
 [ 489 7501]]

Test Confusion Matrix with accuracy 0.6654847947633984
Test Recall: 0.729426433915212
[[1955 1368]
 [1085 2925]]


### SVM Train on 2,3

In [133]:
support_vector_machine23 = SVC(kernel='rbf', gamma=0.1, C=1., random_state=None)
support_vector_machine23.fit(CV23, y23)
print('finished training on 2,3')
#y_scores = clf.decision_function(X_test)
y_scores_train = support_vector_machine23.predict(CV23)
y_scores_test = support_vector_machine23.predict(traincv1)

confusion_train = confusion_matrix(y23, y_scores_train)
confusion_test = confusion_matrix(target1, y_scores_test)


print('Trained on 2,3')
print('Train Confusion Matrix with accuracy {}'.format((confusion_train[0][0]+confusion_train[1][1])/len(y23)))
print('Train Recall: {}'.format((confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]))))
print(confusion_train)
print('')
print('Test Confusion Matrix with accuracy {}'.format((confusion_test[0][0]+confusion_test[1][1])/len(target1)))
print('Test Recall: {}'.format((confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]))))
print(confusion_test)
#precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

finished training on 2,3
Trained on 2,3
Train Confusion Matrix with accuracy 0.919001840867253
Train Recall: 0.937023616143946
[[5980  684]
 [ 504 7499]]

Test Confusion Matrix with accuracy 0.6593481521887359
Test Recall: 0.712784588441331
[[1986 1350]
 [1148 2849]]


### Retrain on Best Model ... 

### Test on TEST Dataset

In [134]:
"""After Eyeballing the model with the best Recall"""
support_vector_machine_best = support_vector_machine12
support_vector_machine_best.fit(X_train, y_train)
print('finished training on FULL Training Data')
#y_scores = clf.decision_function(X_test)
y_scores_train = support_vector_machine_best.predict(X_train)
y_scores_test = support_vector_machine_best.predict(X_test)

confusion_train = confusion_matrix(y_train, y_scores_train)
confusion_test = confusion_matrix(y_test, y_scores_test)


print('Trained on FULL Training Data')
print('Train Confusion Matrix with accuracy {}'.format((confusion_train[0][0]+confusion_train[1][1])/len(y_train)))
print('Train Recall: {}'.format((confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]))))
print(confusion_train)
print('')
print('Test Confusion Matrix with accuracy {}'.format((confusion_test[0][0]+confusion_test[1][1])/len(y_test)))
print('Test Recall: {}'.format((confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]))))
print(confusion_test)
#precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

finished training on FULL Training Data
Trained on FULL Training Data
Train Confusion Matrix with accuracy 0.9084090909090909
Train Recall: 0.9299166666666666
[[ 8826  1174]
 [  841 11159]]

Test Confusion Matrix with accuracy 0.695465746673238
Test Recall: 0.8027
[[6084 4206]
 [1973 8027]]


In [7]:
# grid_values = {'C':[0.01, 0.1, 1, 10], 'gamma':[0.01, 0.1, 1, 10]}
# # default metric to optimize over grid parameters: accuracy
# grid_clf_acc = GridSearchCV(support_vector_machine, param_grid = grid_values, scoring='accuracy')
# #y_decision_fn_scores_acc = grid_clf_acc.decision_function(X_test) 

In [8]:
# """train a model with the best training accuracy"""
# training_accuracy = grid_clf_acc.fit(X_train, y_train)
# best_model = training_accuracy.best_estimator_
# y_scores = best_model.predict(X_test)
# confusion = confusion_matrix(y_test, y_scores)
# #precision, recall, thresholds = precision_recall_curve(y_test, y_scores)
# print(confusion)

In [75]:
# train_scores, test_scores = validation_curve(SVC(), X, y,
#                                             param_name='gamma',
#                                             param_range=param_range, cv=3)

### neural network

### neural network

### neural network

In [181]:
"""split up data into test train spit"""
# X, y = np.matrix(train), np.array(target)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


CV12 = torch.tensor(CV12).float()
y12 = torch.tensor(y12).long()
CV13 = torch.tensor(CV13).float()
y13 = torch.tensor(y13).long()
CV23 = torch.tensor(CV23).float()
y23 = torch.tensor(y23).long()

train = torch.tensor(X_train).float() 
y_train = torch.tensor(y_train).long()
test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).long()



traincv1 = torch.tensor(traincv1).float()
target1 = torch.tensor(target1).long()
traincv2 = torch.tensor(traincv2).float()
target2 = torch.tensor(target2).long()
traincv3 = torch.tensor(traincv3).float()
target3 = torch.tensor(target3).long()
print('data loaded')

tensor(-1.8000)

In [432]:
"""define some functions"""
def train_neural_network(nn_name, learning_rate, epochs, train_tensor, target_tensor): 
    """declare name"""
    name = nn_name
    neural_network = Net(name=name)
    optimizer = optim.Adam(neural_network.parameters(), lr=learning_rate) ## Adam optimizer with parameters and learning rate
    EPOCHS = epochs
    
    XX = torch.tensor(train_tensor).float()
    target = torch.tensor(target_tensor).long()
    
    for epoch in range(EPOCHS):
        XX = XX
        target = target
        
        neural_network.zero_grad()
        output = neural_network.forward(XX)
        predicted = output.float()
        
        #print(predicted.shape, target.shape)
        loss = F.cross_entropy(predicted, target)
        #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
        #F.cross_entropy
        #  F.leaky_relu
        loss.backward() ## do the backprop
        optimizer.step() ## update the weights
        if epoch % 200 == 0:
            print('epoch {} had loss {}'.format(epoch, loss))

    print('final loss was {}\n\n'.format(loss))
    
    return neural_network



#####
##########
###------------########
######-----------------##
#####---------------------######


def test_scores(neural_network, X_train, y_train, X_test, y_test):
    total_train = 0
    correct_train = 0
    total_test = 0
    correct_test = 0
    nn_output_train = []
    nn_output_test = []

    with torch.no_grad(): 
        XX_train = torch.tensor(X_train).float()
        XX_test = torch.tensor(X_test).float()    
        #target = torch.tensor(target_tensor).long() 

        #yy = y_train
        output_train = neural_network.forward(XX_train)
        output_test = neural_network.forward(XX_test)

        for idx, i in enumerate(output_train):
    #         print(int(torch.argmax(i)))
    #         print(type(torch.argmax(i)))
    #        print(type(output_train))
            nn_output_train.append(int(torch.argmax(i)))
            if torch.argmax(i) == y_train[idx]:
                correct_train += 1 ## check the predicted value with the actual value
            total_train += 1

        for idx, i in enumerate(output_test):
            nn_output_test.append(int(torch.argmax(i)))
            if torch.argmax(i) == y_test[idx]:
                correct_test += 1 ## check the predicted value with the actual value
            total_test += 1


    y_scores_train = np.array(nn_output_train)
    y_scores_test = np.array(nn_output_test)


    # print('the accuracy on the training data is is', round(correct_train/total_train, 4))
    # print(confusion_matrix(nn_output_train, y13))
    # print('the accuracy on the testing data is is', round(correct_test/total_test, 4))
    # print(confusion_matrix(nn_output_test, target2))

    confusion_train = confusion_matrix(y13, y_scores_train)
    confusion_test = confusion_matrix(target2, y_scores_test)


    print('Trained')
    train_accuracy = (confusion_train[0][0]+confusion_train[1][1])/len(y_train)
    train_recall = confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1])
    train_precision = confusion_train[1][1]/(confusion_train[0][1]+confusion_train[1][1])
                                             
    test_accuracy = (confusion_test[0][0]+confusion_test[1][1])/len(y_test)
    test_recall = confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1])
    test_precision = confusion_test[1][1]/(confusion_test[0][1]+confusion_test[1][1])
                                                                                    
    print('Train Accuracy: {}'.format(train_accuracy))
    print('Train Recall: {}'.format(train_recall))
    print('Train Precision: {}'.format(train_precision))
    print(confusion_train)
    
    print('')
    print('Test Accuracy: {}'.format(test_accuracy))
    print('Test Recall: {}'.format(test_recall))
    print('Test Precision: {}'.format(test_precision))
    print(confusion_test)
    
    return_dic = {'Train_Accuracy': train_accuracy, 'Train_Recall': train_recall, 'Train_Precision': train_precision, 
                 'Test_Accuracy': test_accuracy, 'Test_Recall': test_recall, 'Test_Precision': test_precision}
                                           
                                           
    return confusion_train, confusion_test, return_dic


In [433]:
"""in order to make a neural network, we need to create an object belonging to the NEURAL NETWORK class"""

class Net(nn.Module):
    
    def __init__(self, name=None):
#         """inherit NN methods, attributes, functions, etc..., without this we can't use it"""
        super().__init__()
        self.fc1 = nn.Linear(in_features=1*X_train.shape[1], out_features=128) ## (each input is a 2x1 vector) 
        self.fc2 = nn.Linear(in_features=128, out_features=128) ## (layer1 output, layer 2 output)        
        self.fc3 = nn.Linear(in_features=128, out_features=2) ## (layer2 output, layer 3 output)
        #self.fc4 = nn.Linear(in_features=64, out_features=2) ## (layer 3 output, layer 4 output)
        self.name = None
        self.train_size = len(y_train)
        self.test_size = len(y_test)
        print('the nnn name is {}'.format(self.name))
        
        
    def forward(self, x):
#         x = F.relu(self.fc1(x)) ## pass vector into first layer
#         x = F.relu(self.fc2(x)) ## 2nd layer
#         x = F.relu(self.fc3(x)) ### 3rd layer
        #x = F.relu
        x = F.leaky_relu(self.fc1(x), inplace=True, negative_slope=0.9)
        x = F.leaky_relu(self.fc2(x), inplace=True, negative_slope=0.9)
        x = F.leaky_relu(self.fc3(x), inplace=True, negative_slope=0.9)
        #x = self.fc4(x) ### and then run softmax regression on the final layer to estimate 0,...,9 
        y = F.softmax(x, dim=1)
        #y = F.log_softmax(x, dim=1)
        return  y
                


In [435]:
"""Train 12"""
nn12 = train_neural_network(nn_name='NN12_0', learning_rate=0.001, epochs=100, train_tensor=CV12, target_tensor=y12)
c12_train, c12_test, c12_scores = test_scores(neural_network=nn12, X_train=CV12, y_train=y12, X_test=traincv3, y_test=target3)

"""Train 13"""
nn13 = train_neural_network(nn_name='NN13_0', learning_rate=0.001, epochs=100, train_tensor=CV13, target_tensor=y13)
c13_train, c13_test, c13_scores = test_scores(neural_network=nn13, X_train=CV13, y_train=y13, X_test=traincv2, y_test=target2)

"""Train 23"""
nn23 = train_neural_network(nn_name='NN23_0', learning_rate=0.001, epochs=100, train_tensor=CV23, target_tensor=y23)
c23_train, c23_test, c23_scores = test_scores(neural_network=nn23, X_train=CV23, y_train=y23, X_test=traincv1, y_test=target1)

the nnn name is None
epoch 0 had loss 0.6887909770011902
final loss was 0.6041503548622131




ValueError: Found input variables with inconsistent numbers of samples: [14667, 14666]

In [ ]:
"""Train the Best NN"""



In [218]:
"""COPY OF NN"""

# """declare neural networks"""
# neural_network12 = Net(name='net12')

# def train_12():    
#     optimizer12 = optim.Adam(neural_network12.parameters(), lr=0.001) ## Adam optimizer with parameters and learning rate

#     EPOCHS = 1200 
#     for epoch in range(EPOCHS):
#         XX = CV12
#         neural_network12.zero_grad()
#         output = neural_network12.forward(XX)
#         predicted = output.float()
#         #yy = y_train
#         target = y12
#         #print(predicted.shape, target.shape)
#         loss = F.cross_entropy(predicted, target)
#         #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
#         #F.cross_entropy
#         #  F.leaky_relu
#         loss.backward() ## do the backprop
#         optimizer12.step() ## update the weights
#         if epoch % 200 == 0:
#             print('epoch {} had loss {}'.format(epoch, loss))

#     print('final loss was {}\n\n'.format(loss))


#     ########
#     ###########
#     #########------Test NN-------------####
#     ########
#     ##############
#     #########

#     """Now Test on 3rd Dataset"""
#     """aaa"""

#     total_train = 0
#     correct_train = 0
#     total_test = 0
#     correct_test = 0

#     nn_output_train = []
#     nn_output_test = []

#     with torch.no_grad(): 
#         XX_train = CV12
#         XX_test = traincv3    

#         #yy = y_train
#         output_train = neural_network12.forward(XX_train)
#         output_test = neural_network12.forward(XX_test)

#         for idx, i in enumerate(output_train):
#     #         print(int(torch.argmax(i)))
#     #         print(type(torch.argmax(i)))
#     #        print(type(output_train))
#             nn_output_train.append(int(torch.argmax(i)))
#             if torch.argmax(i) == y12[idx]:
#                 correct_train += 1 ## check the predicted value with the actual value
#             total_train += 1

#         for idx, i in enumerate(output_test):
#             nn_output_test.append(int(torch.argmax(i)))
#             if torch.argmax(i) == target3[idx]:
#                 correct_test += 1 ## check the predicted value with the actual value
#             total_test += 1


#     y_scores_train = np.array(nn_output_train)
#     y_scores_test = np.array(nn_output_test)

#     # print('the accuracy on the training data is is', round(correct_train/total_train, 4))
#     # print(confusion_matrix(nn_output_train, y12))
#     # print('the accuracy on the testing data is is', round(correct_test/total_test, 4))
#     # print(confusion_matrix(nn_output_test, target3))
#     confusion_train = confusion_matrix(y12, y_scores_train)
#     confusion_test = confusion_matrix(target3, y_scores_test)


#     print('Trained on 1,2')
#     train_accuracy = confusion_train[0][0]+confusion_train[1][1])/len(y23)
#     train_recall = confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]
#     train_precision = confusion_train[1][1]/(confusion_train[0][1]+confusion_train[1][1]
                                             
#     test_accuracy = confusion_test[0][0]+confusion_test[1][1])/len(target1)
#     test_recall = confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]
#     test_precision = confusion_test[1][1]/(confusion_test[0][1]+confusion_test[1][1]
                                                                                    
#     print('Train Accuracy: {}'.format(train_accuracy))
#     print('Train Recall: {}'.format(train_recall))
#     print('Train Precision: {}'.format(train_precision))
#     print(confusion_train)
    
#     print('')
#     print('Test Accuracy: {}'.format(test_accuracy))
#     print('Test Recall: {}'.format(test_recall))
#     print('Test Precision: {}'.format(test_precision))
#     print(confusion_test)
    
#     return_dic = {'Train_Accuracy': train_accuracy, 'Train_Recall': train_recall, 'Train_Precision': train_precision, 
#                  'Test_Accuracy': test_accuracy, 'Test_Recall': test_recall, 'Test_Precision': test_precision}
                                           
                                           
#     return neural_network12, confusion_train, confusion_test, return_dic

20290

### NN Train on 1,2

In [196]:
"""declare neural networks"""
neural_network12 = Net(name='net12')

def train_12():    
    optimizer12 = optim.Adam(neural_network12.parameters(), lr=0.001) ## Adam optimizer with parameters and learning rate

    EPOCHS = 1200 
    for epoch in range(EPOCHS):
        XX = CV12
        neural_network12.zero_grad()
        output = neural_network12.forward(XX)
        predicted = output.float()
        #yy = y_train
        target = y12
        #print(predicted.shape, target.shape)
        loss = F.cross_entropy(predicted, target)
        #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
        #F.cross_entropy
        #  F.leaky_relu
        loss.backward() ## do the backprop
        optimizer12.step() ## update the weights
        if epoch % 200 == 0:
            print('epoch {} had loss {}'.format(epoch, loss))

    print('final loss was {}\n\n'.format(loss))


    ########
    ###########
    #########------Test NN-------------####
    ########
    ##############
    #########

    """Now Test on 3rd Dataset"""
    """aaa"""

    total_train = 0
    correct_train = 0
    total_test = 0
    correct_test = 0

    nn_output_train = []
    nn_output_test = []

    with torch.no_grad(): 
        XX_train = CV12
        XX_test = traincv3    

        #yy = y_train
        output_train = neural_network12.forward(XX_train)
        output_test = neural_network12.forward(XX_test)

        for idx, i in enumerate(output_train):
    #         print(int(torch.argmax(i)))
    #         print(type(torch.argmax(i)))
    #        print(type(output_train))
            nn_output_train.append(int(torch.argmax(i)))
            if torch.argmax(i) == y12[idx]:
                correct_train += 1 ## check the predicted value with the actual value
            total_train += 1

        for idx, i in enumerate(output_test):
            nn_output_test.append(int(torch.argmax(i)))
            if torch.argmax(i) == target3[idx]:
                correct_test += 1 ## check the predicted value with the actual value
            total_test += 1


    y_scores_train = np.array(nn_output_train)
    y_scores_test = np.array(nn_output_test)

    # print('the accuracy on the training data is is', round(correct_train/total_train, 4))
    # print(confusion_matrix(nn_output_train, y12))
    # print('the accuracy on the testing data is is', round(correct_test/total_test, 4))
    # print(confusion_matrix(nn_output_test, target3))
    confusion_train = confusion_matrix(y12, y_scores_train)
    confusion_test = confusion_matrix(target3, y_scores_test)


    print('Trained on 1,2')
    train_accuracy = confusion_train[0][0]+confusion_train[1][1])/len(y23)
    train_recall = confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]
    train_precision = confusion_train[1][1]/(confusion_train[0][1]+confusion_train[1][1]
                                             
    test_accuracy = confusion_test[0][0]+confusion_test[1][1])/len(target1)
    test_recall = confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]
    test_precision = confusion_test[1][1]/(confusion_test[0][1]+confusion_test[1][1]
                                                                                    
    print('Train Accuracy: {}'.format(train_accuracy))
    print('Train Recall: {}'.format(train_recall))
    print('Train Precision: {}'.format(train_precision))
    print(confusion_train)
    
    print('')
    print('Test Accuracy: {}'.format(test_accuracy))
    print('Test Recall: {}'.format(test_recall))
    print('Test Precision: {}'.format(test_precision))
    print(confusion_test)
    
    return_dic = {'Train_Accuracy': train_accuracy, 'Train_Recall': train_recall, 'Train_Precision': train_precision, 
                 'Test_Accuracy': test_accuracy, 'Test_Recall': test_recall, 'Test_Precision': test_precision}
                                           
                                           
    return neural_network12, confusion_train, confusion_test, return_dic

epoch 0 had loss 0.6853665113449097
epoch 200 had loss 0.6028300523757935
epoch 400 had loss 0.6005607843399048
epoch 600 had loss 0.5971062183380127
epoch 800 had loss 0.595834493637085
epoch 1000 had loss 0.5890336036682129
final loss was 0.586561381816864


Trained on 1,2
Train Confusion Matrix with accuracy 0.7130778671757807
Train Recall: 0.7573373298363931
[[4394 2265]
 [1943 6064]]

Test Confusion Matrix with accuracy 0.6888464685028634
Test Recall: 0.7390433258201853
[[2101 1240]
 [1042 2951]]


### NN Train on 1,3

In [415]:
def train_neural_network(nn_name, learning_rate, epochs, train_tensor, target_tensor): 
    """declare name"""
    neural_network = Net(name=nn_name)
    optimizer = optim.Adam(neural_network.parameters(), lr=learning_rate) ## Adam optimizer with parameters and learning rate
    EPOCHS = epochs
    
    XX = torch.tensor(train_tensor).float()
    target = torch.tensor(target_tensor).long()
    
    for epoch in range(EPOCHS):
        XX = XX
        target = target
        
        neural_network.zero_grad()
        output = neural_network.forward(XX)
        predicted = output.float()
        
        #print(predicted.shape, target.shape)
        loss = F.cross_entropy(predicted, target)
        #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
        #F.cross_entropy
        #  F.leaky_relu
        loss.backward() ## do the backprop
        optimizer.step() ## update the weights
        if epoch % 200 == 0:
            print('epoch {} had loss {}'.format(epoch, loss))

    print('final loss was {}\n\n'.format(loss))
    
    return neural_network


In [416]:
samp_nn = train_neural_network('franlin',0.001, 100, CV13, y13)

the nnn name is None
epoch 0 had loss 0.6935456991195679
final loss was 0.6042400598526001




In [428]:
def test_scores(neural_network, X_train, y_train, X_test, y_test):
    total_train = 0
    correct_train = 0
    total_test = 0
    correct_test = 0
    nn_output_train = []
    nn_output_test = []

    with torch.no_grad(): 
        XX_train = torch.tensor(X_train).float()
        XX_test = torch.tensor(X_test).float()    
        #target = torch.tensor(target_tensor).long() 

        #yy = y_train
        output_train = neural_network.forward(XX_train)
        output_test = neural_network.forward(XX_test)

        for idx, i in enumerate(output_train):
    #         print(int(torch.argmax(i)))
    #         print(type(torch.argmax(i)))
    #        print(type(output_train))
            nn_output_train.append(int(torch.argmax(i)))
            if torch.argmax(i) == y_train[idx]:
                correct_train += 1 ## check the predicted value with the actual value
            total_train += 1

        for idx, i in enumerate(output_test):
            nn_output_test.append(int(torch.argmax(i)))
            if torch.argmax(i) == y_test[idx]:
                correct_test += 1 ## check the predicted value with the actual value
            total_test += 1


    y_scores_train = np.array(nn_output_train)
    y_scores_test = np.array(nn_output_test)


    # print('the accuracy on the training data is is', round(correct_train/total_train, 4))
    # print(confusion_matrix(nn_output_train, y13))
    # print('the accuracy on the testing data is is', round(correct_test/total_test, 4))
    # print(confusion_matrix(nn_output_test, target2))

    confusion_train = confusion_matrix(y13, y_scores_train)
    confusion_test = confusion_matrix(target2, y_scores_test)


    print('Trained')
    train_accuracy = (confusion_train[0][0]+confusion_train[1][1])/len(y_train)
    train_recall = confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1])
    train_precision = confusion_train[1][1]/(confusion_train[0][1]+confusion_train[1][1])
                                             
    test_accuracy = (confusion_test[0][0]+confusion_test[1][1])/len(y_test)
    test_recall = confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1])
    test_precision = confusion_test[1][1]/(confusion_test[0][1]+confusion_test[1][1])
                                                                                    
    print('Train Accuracy: {}'.format(train_accuracy))
    print('Train Recall: {}'.format(train_recall))
    print('Train Precision: {}'.format(train_precision))
    print(confusion_train)
    
    print('')
    print('Test Accuracy: {}'.format(test_accuracy))
    print('Test Recall: {}'.format(test_recall))
    print('Test Precision: {}'.format(test_precision))
    print(confusion_test)
    
    return_dic = {'Train_Accuracy': train_accuracy, 'Train_Recall': train_recall, 'Train_Precision': train_precision, 
                 'Test_Accuracy': test_accuracy, 'Test_Recall': test_recall, 'Test_Precision': test_precision}
                                           
                                           
    return neural_network, confusion_train, confusion_test, return_dic

In [ ]:
# # """declare name"""
# # neural_network13 = Net(name='net13')

# def train_13():
#     """declare name"""
#     neural_network = Net(name='net13')
#     print('Neural network name is {}'.format(neural_network.name))
#     optimizer = optim.Adam(neural_network.parameters(), lr=0.001) ## Adam optimizer with parameters and learning rate
#     EPOCHS = 1200
    
    
#     for epoch in range(EPOCHS):
#         XX = CV13
#         neural_network.zero_grad()
#         output = neural_network.forward(XX)
#         predicted = output.float()
#         #yy = y_train
#         target = y13
#         #print(predicted.shape, target.shape)
#         loss = F.cross_entropy(predicted, target)
#         #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
#         #F.cross_entropy
#         #  F.leaky_relu
#         loss.backward() ## do the backprop
#         optimizer.step() ## update the weights
#         if epoch % 200 == 0:
#             print('epoch {} had loss {}'.format(epoch, loss))

#     print('final loss was {}\n\n'.format(loss))


#     ########
#     ###########
#     #########------Test NN-------------####
#     ########
#     ##############
#     #########


#     """Now Test on 2nd Dataset"""
#     """aaa"""

#     total_train = 0
#     correct_train = 0
#     total_test = 0
#     correct_test = 0

#     nn_output_train = []
#     nn_output_test = []

#     with torch.no_grad(): 
#         XX_train = CV13
#         XX_test = traincv2    

#         #yy = y_train
#         output_train = neural_network13.forward(XX_train)
#         output_test = neural_network13.forward(XX_test)

#         for idx, i in enumerate(output_train):
#     #         print(int(torch.argmax(i)))
#     #         print(type(torch.argmax(i)))
#     #        print(type(output_train))
#             nn_output_train.append(int(torch.argmax(i)))
#             if torch.argmax(i) == y13[idx]:
#                 correct_train += 1 ## check the predicted value with the actual value
#             total_train += 1

#         for idx, i in enumerate(output_test):
#             nn_output_test.append(int(torch.argmax(i)))
#             if torch.argmax(i) == target2[idx]:
#                 correct_test += 1 ## check the predicted value with the actual value
#             total_test += 1


#     y_scores_train = np.array(nn_output_train)
#     y_scores_test = np.array(nn_output_test)


#     # print('the accuracy on the training data is is', round(correct_train/total_train, 4))
#     # print(confusion_matrix(nn_output_train, y13))
#     # print('the accuracy on the testing data is is', round(correct_test/total_test, 4))
#     # print(confusion_matrix(nn_output_test, target2))

#     confusion_train = confusion_matrix(y13, y_scores_train)
#     confusion_test = confusion_matrix(target2, y_scores_test)


#     print('Trained on 1,3')
#     train_accuracy = confusion_train[0][0]+confusion_train[1][1])/len(y23)
#     train_recall = confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]
#     train_precision = confusion_train[1][1]/(confusion_train[0][1]+confusion_train[1][1]
                                             
#     test_accuracy = confusion_test[0][0]+confusion_test[1][1])/len(target1)
#     test_recall = confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]
#     test_precision = confusion_test[1][1]/(confusion_test[0][1]+confusion_test[1][1]
                                                                                    
#     print('Train Accuracy: {}'.format(train_accuracy))
#     print('Train Recall: {}'.format(train_recall))
#     print('Train Precision: {}'.format(train_precision))
#     print(confusion_train)
    
#     print('')
#     print('Test Accuracy: {}'.format(test_accuracy))
#     print('Test Recall: {}'.format(test_recall))
#     print('Test Precision: {}'.format(test_precision))
#     print(confusion_test)
    
#     return_dic = {'Train_Accuracy': train_accuracy, 'Train_Recall': train_recall, 'Train_Precision': train_precision, 
#                  'Test_Accuracy': test_accuracy, 'Test_Recall': test_recall, 'Test_Precision': test_precision}
                                           
                                           
#     return neural_network13, confusion_train, confusion_test, return_dic

In [391]:
# # """declare name"""
# # neural_network13 = Net(name='net13')

# def train_13():
#     """declare name"""
#     neural_network = Net(name='net13')
#     print('Neural network name is {}'.format(neural_network.name))
#     optimizer = optim.Adam(neural_network.parameters(), lr=0.001) ## Adam optimizer with parameters and learning rate
#     EPOCHS = 1200
    
    
#     for epoch in range(EPOCHS):
#         XX = CV13
#         neural_network.zero_grad()
#         output = neural_network13.forward(XX)
#         predicted = output.float()
#         #yy = y_train
#         target = y13
#         #print(predicted.shape, target.shape)
#         loss = F.cross_entropy(predicted, target)
#         #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
#         #F.cross_entropy
#         #  F.leaky_relu
#         loss.backward() ## do the backprop
#         optimizer13.step() ## update the weights
#         if epoch % 200 == 0:
#             print('epoch {} had loss {}'.format(epoch, loss))

#     print('final loss was {}\n\n'.format(loss))


#     ########
#     ###########
#     #########------Test NN-------------####
#     ########
#     ##############
#     #########


#     """Now Test on 2nd Dataset"""
#     """aaa"""

#     total_train = 0
#     correct_train = 0
#     total_test = 0
#     correct_test = 0

#     nn_output_train = []
#     nn_output_test = []

#     with torch.no_grad(): 
#         XX_train = CV13
#         XX_test = traincv2    

#         #yy = y_train
#         output_train = neural_network13.forward(XX_train)
#         output_test = neural_network13.forward(XX_test)

#         for idx, i in enumerate(output_train):
#     #         print(int(torch.argmax(i)))
#     #         print(type(torch.argmax(i)))
#     #        print(type(output_train))
#             nn_output_train.append(int(torch.argmax(i)))
#             if torch.argmax(i) == y13[idx]:
#                 correct_train += 1 ## check the predicted value with the actual value
#             total_train += 1

#         for idx, i in enumerate(output_test):
#             nn_output_test.append(int(torch.argmax(i)))
#             if torch.argmax(i) == target2[idx]:
#                 correct_test += 1 ## check the predicted value with the actual value
#             total_test += 1


#     y_scores_train = np.array(nn_output_train)
#     y_scores_test = np.array(nn_output_test)


#     # print('the accuracy on the training data is is', round(correct_train/total_train, 4))
#     # print(confusion_matrix(nn_output_train, y13))
#     # print('the accuracy on the testing data is is', round(correct_test/total_test, 4))
#     # print(confusion_matrix(nn_output_test, target2))

#     confusion_train = confusion_matrix(y13, y_scores_train)
#     confusion_test = confusion_matrix(target2, y_scores_test)


#     print('Trained on 1,3')
#     train_accuracy = confusion_train[0][0]+confusion_train[1][1])/len(y23)
#     train_recall = confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]
#     train_precision = confusion_train[1][1]/(confusion_train[0][1]+confusion_train[1][1]
                                             
#     test_accuracy = confusion_test[0][0]+confusion_test[1][1])/len(target1)
#     test_recall = confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]
#     test_precision = confusion_test[1][1]/(confusion_test[0][1]+confusion_test[1][1]
                                                                                    
#     print('Train Accuracy: {}'.format(train_accuracy))
#     print('Train Recall: {}'.format(train_recall))
#     print('Train Precision: {}'.format(train_precision))
#     print(confusion_train)
    
#     print('')
#     print('Test Accuracy: {}'.format(test_accuracy))
#     print('Test Recall: {}'.format(test_recall))
#     print('Test Precision: {}'.format(test_precision))
#     print(confusion_test)
    
#     return_dic = {'Train_Accuracy': train_accuracy, 'Train_Recall': train_recall, 'Train_Precision': train_precision, 
#                  'Test_Accuracy': test_accuracy, 'Test_Recall': test_recall, 'Test_Precision': test_precision}
                                           
                                           
#     return neural_network13, confusion_train, confusion_test, return_dic

### NN Train 2,3

In [ ]:
"""delcare name"""
neural_network23 = Net(name='net23')

def train_23():
    optimizer23 = optim.Adam(neural_network23.parameters(), lr=0.001) ## Adam optimizer with parameters and learning rate

    EPOCHS = 1200 
    for epoch in range(EPOCHS):
        XX = CV23
        neural_network23.zero_grad()
        output = neural_network23.forward(XX)
        predicted = output.float()
        #yy = y_train
        target = y23
        #print(predicted.shape, target.shape)
        loss = F.cross_entropy(predicted, target)
        #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
        #F.cross_entropy
        #  F.leaky_relu
        loss.backward() ## do the backprop
        optimizer23.step() ## update the weights
        if epoch % 200 == 0:
            print('epoch {} had loss {}'.format(epoch, loss))

    print('final loss was {}\n\n'.format(loss))

    ########
    ###########
    #########------Test NN-------------####
    ########
    ##############
    #########
    """Now Test on 1st Dataset"""
    """aaa"""

    total_train = 0
    correct_train = 0
    total_test = 0
    correct_test = 0

    nn_output_train = []
    nn_output_test = []

    with torch.no_grad(): 
        XX_train = CV23
        XX_test = traincv1    

        #yy = y_train
        output_train = neural_network23.forward(XX_train)
        output_test = neural_network23.forward(XX_test)

        for idx, i in enumerate(output_train):
    #         print(int(torch.argmax(i)))
    #         print(type(torch.argmax(i)))
    #        print(type(output_train))
            nn_output_train.append(int(torch.argmax(i)))
            if torch.argmax(i) == y23[idx]:
                correct_train += 1 ## check the predicted value with the actual value
            total_train += 1

        for idx, i in enumerate(output_test):
            nn_output_test.append(int(torch.argmax(i)))
            if torch.argmax(i) == target1[idx]:
                correct_test += 1 ## check the predicted value with the actual value
            total_test += 1


    y_scores_train = np.array(nn_output_train)
    y_scores_test = np.array(nn_output_test)

    # print('the accuracy on the training data is is', round(correct_train/total_train, 4))
    # print(confusion_matrix(nn_output_train, y23))
    # print('the accuracy on the testing data is is', round(correct_test/total_test, 4))
    # print(confusion_matrix(nn_output_test, target1))


    confusion_train = confusion_matrix(y23, y_scores_train)
    confusion_test = confusion_matrix(target1, y_scores_test)


    print('Trained on 2,3')
    train_accuracy = confusion_train[0][0]+confusion_train[1][1])/len(y23)
    train_recall = confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]
    train_precision = confusion_train[1][1]/(confusion_train[0][1]+confusion_train[1][1]
                                             
    test_accuracy = confusion_test[0][0]+confusion_test[1][1])/len(target1)
    test_recall = confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]
    test_precision = confusion_test[1][1]/(confusion_test[0][1]+confusion_test[1][1]
                                                                                    
    print('Train Accuracy: {}'.format(train_accuracy))
    print('Train Recall: {}'.format(train_recall))
    print('Train Precision: {}'.format(train_precision))
    print(confusion_train)
    
    print('')
    print('Test Accuracy: {}'.format(test_accuracy))
    print('Test Recall: {}'.format(test_recall))
    print('Test Precision: {}'.format(test_precision))
    print(confusion_test)
    
    return_dic = {'Train_Accuracy': train_accuracy, 'Train_Recall': train_recall, 'Train_Precision': train_precision, 
                 'Test_Accuracy': test_accuracy, 'Test_Recall': test_recall, 'Test_Precision': test_precision}
                                           
                                           
    return neural_network23, confusion_train, confusion_test, return_dic

In [392]:
##PRACITC
# """delcare name"""
# neural_network23 = Net(name='net23')

# def train_23():
#     optimizer23 = optim.Adam(neural_network23.parameters(), lr=0.001) ## Adam optimizer with parameters and learning rate

#     EPOCHS = 1200 
#     for epoch in range(EPOCHS):
#         XX = CV23
#         neural_network23.zero_grad()
#         output = neural_network23.forward(XX)
#         predicted = output.float()
#         #yy = y_train
#         target = y23
#         #print(predicted.shape, target.shape)
#         loss = F.cross_entropy(predicted, target)
#         #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
#         #F.cross_entropy
#         #  F.leaky_relu
#         loss.backward() ## do the backprop
#         optimizer23.step() ## update the weights
#         if epoch % 200 == 0:
#             print('epoch {} had loss {}'.format(epoch, loss))

#     print('final loss was {}\n\n'.format(loss))

#     ########
#     ###########
#     #########------Test NN-------------####
#     ########
#     ##############
#     #########
#     """Now Test on 1st Dataset"""
#     """aaa"""

#     total_train = 0
#     correct_train = 0
#     total_test = 0
#     correct_test = 0

#     nn_output_train = []
#     nn_output_test = []

#     with torch.no_grad(): 
#         XX_train = CV23
#         XX_test = traincv1    

#         #yy = y_train
#         output_train = neural_network23.forward(XX_train)
#         output_test = neural_network23.forward(XX_test)

#         for idx, i in enumerate(output_train):
#     #         print(int(torch.argmax(i)))
#     #         print(type(torch.argmax(i)))
#     #        print(type(output_train))
#             nn_output_train.append(int(torch.argmax(i)))
#             if torch.argmax(i) == y23[idx]:
#                 correct_train += 1 ## check the predicted value with the actual value
#             total_train += 1

#         for idx, i in enumerate(output_test):
#             nn_output_test.append(int(torch.argmax(i)))
#             if torch.argmax(i) == target1[idx]:
#                 correct_test += 1 ## check the predicted value with the actual value
#             total_test += 1


#     y_scores_train = np.array(nn_output_train)
#     y_scores_test = np.array(nn_output_test)

#     # print('the accuracy on the training data is is', round(correct_train/total_train, 4))
#     # print(confusion_matrix(nn_output_train, y23))
#     # print('the accuracy on the testing data is is', round(correct_test/total_test, 4))
#     # print(confusion_matrix(nn_output_test, target1))


#     confusion_train = confusion_matrix(y23, y_scores_train)
#     confusion_test = confusion_matrix(target1, y_scores_test)


#     print('Trained on 2,3')
#     train_accuracy = confusion_train[0][0]+confusion_train[1][1])/len(y23)
#     train_recall = confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]
#     train_precision = confusion_train[1][1]/(confusion_train[0][1]+confusion_train[1][1]
                                             
#     test_accuracy = confusion_test[0][0]+confusion_test[1][1])/len(target1)
#     test_recall = confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]
#     test_precision = confusion_test[1][1]/(confusion_test[0][1]+confusion_test[1][1]
                                                                                    
#     print('Train Accuracy: {}'.format(train_accuracy))
#     print('Train Recall: {}'.format(train_recall))
#     print('Train Precision: {}'.format(train_precision))
#     print(confusion_train)
    
#     print('')
#     print('Test Accuracy: {}'.format(test_accuracy))
#     print('Test Recall: {}'.format(test_recall))
#     print('Test Precision: {}'.format(test_precision))
#     print(confusion_test)
    
#     return_dic = {'Train_Accuracy': train_accuracy, 'Train_Recall': train_recall, 'Train_Precision': train_precision, 
#                  'Test_Accuracy': test_accuracy, 'Test_Recall': test_recall, 'Test_Precision': test_precision}
                                           
                                           
#     return neural_network23, confusion_train, confusion_test, return_dic

### Retrain with Best Model

In [342]:
#best_models_df = pd.DataFrame(columns=['NN', 'HL1', 'HL2', 'HL3', 'HL4', 'HL5', 'TN', 'FP', 'FN', 'TP'])
sample_nn = {'NN': 0, 'LR': 0, 'Layers': 0, 'Total_Layers': 0,
             'Train_TN': 0, 'Train_TP':0, 'Train_FP':0, 'Train_FN':0, 
             'Test_TN': 0, 'Test_TP':0, 'Test_FP':0, 'Test_FN':0}

sample_nn = pd.DataFrame([sample_nn])

best_models_df = sample_nn.copy()
best_models_df

In [200]:
neural_network_best = neural_network23
"""NN, """
nn_name = neural_network_best.name
learning_rate = 0.001
n_layers = int(str(neural_network_best).split('fc')[-1].split(')')[0])


optimizer_best = optim.Adam(neural_network_best.parameters(), lr=learning_rate) ## Adam optimizer with parameters and learning rate




EPOCHS = 1200 
for epoch in range(EPOCHS):
    XX = train
    neural_network_best.zero_grad()
    output = neural_network_best.forward(XX)
    predicted = output.float()
    #yy = y_train
    target = y_train
    #print(predicted.shape, target.shape)
    loss = F.cross_entropy(predicted, target)
    #loss = F.nll_loss(output, y) ## calculate loss function with MSE or cross_entropy
    #F.cross_entropy
    #  F.leaky_relu
    loss.backward() ## do the backprop
    optimizer_best.step() ## update the weights
    if epoch % 200 == 0:
        print('epoch {} had loss {}'.format(epoch, loss))

print('final loss was {}\n\n'.format(loss))


########
###########
#########------Test NN-------------####
########
##############
#########


"""Now Test on FULL Training Dataset"""
"""aaa"""

total_train = 0
correct_train = 0
total_test = 0
correct_test = 0

nn_output_train = []
nn_output_test = []

with torch.no_grad(): 
    XX_train = train
    XX_test = test    
    
    #input the matrix and output the tensor of classifications
    output_train = neural_network_best.forward(XX_train)
    output_test = neural_network_best.forward(XX_test)
    
    for idx, i in enumerate(output_train):
#         print(int(torch.argmax(i)))
#         print(type(torch.argmax(i)))
#        print(type(output_train))
        nn_output_train.append(int(torch.argmax(i)))
        if torch.argmax(i) == y_train[idx]:
            correct_train += 1 ## check the predicted value with the actual value
        total_train += 1

    for idx, i in enumerate(output_test):
        nn_output_test.append(int(torch.argmax(i)))
        if torch.argmax(i) == y_test[idx]:
            correct_test += 1 ## check the predicted value with the actual value
        total_test += 1
            

y_scores_train = np.array(nn_output_train)
y_scores_test = np.array(nn_output_test)


# print('the accuracy on the training data is is', round(correct_train/total_train, 4))
# print(confusion_matrix(nn_output_train, y13))
# print('the accuracy on the testing data is is', round(correct_test/total_test, 4))
# print(confusion_matrix(nn_output_test, target2))

confusion_train = confusion_matrix(y_train, y_scores_train)
confusion_test = confusion_matrix(y_test, y_scores_test)


print('Trained on FULL Training Data')
print('Train Confusion Matrix with accuracy {}'.format((confusion_train[0][0]+confusion_train[1][1])/len(y_train)))
print('Train Recall: {}'.format((confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]))))
print(confusion_train)
print('')
print('Test Confusion Matrix with accuracy {}'.format((confusion_test[0][0]+confusion_test[1][1])/len(y_test)))
print('Test Recall: {}'.format((confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]))))
print(confusion_test)


#########
###########
########-----Append the data into a new dictionary----##
###########
#############
#########
"""append dictionary to dataframe"""

nn_data = {'NN': 0, 'LR': 0, 'Layers': 0, 'Total_Layers': 0,
             'Train_TN': 0, 'Train_TP':0, 'Train_FP':0, 'Train_FN':0, 
             'Test_TN': 0, 'Test_TP':0, 'Test_FP':0, 'Test_FN':0}

nn_data['NN'] = nn_name
nn_data['LR'] = learning_rate
nn_data['Total_Layers']= n_layers

layer_list = np.array([[]])
for hidden_layer_index in range(n_layers): 
    arc = str(neural_network_best).split('fc')[1:][hidden_layer_index].split(',')
    in_layer = int(arc[0].split(': ')[1].split('=')[1])
    out_layer = int(arc[1].split('=')[1])
    layer_values = np.array([in_layer, out_layer])
    layer_list = np.append(arr=[layer_list], values=[layer_values])

nn_data['Layers'] = layer_list

nn_data['Train_TN'] = confusion_train[0][0]
nn_data['Train_TP'] = confusion_train[1][1]
nn_data['Train_FP'] = confusion_train[0][1]
nn_data['Train_FN'] = confusion_train[1][0]

nn_data['Test_TN'] = confusion_test[0][0]
nn_data['Test_TP'] = confusion_test[1][1]
nn_data['Test_FP'] = confusion_test[0][1]
nn_data['Test_FN'] = confusion_test[1][0]

nn_data = pd.DataFrame([nn_data])

best_models_df = pd.concat([best_models_df, nn_data])

#np.array(nn_data['Layers'])

best_models_df

epoch 0 had loss 0.5885718464851379
epoch 200 had loss 0.5964018702507019
epoch 400 had loss 0.5847443342208862
epoch 600 had loss 0.5839317440986633
epoch 800 had loss 0.5869135856628418
epoch 1000 had loss 0.5872578024864197
final loss was 0.5808111429214478


Trained on FULL Training Data
Train Confusion Matrix with accuracy 0.7222272727272727
Train Recall: 0.746
[[6937 3063]
 [3048 8952]]

Test Confusion Matrix with accuracy 0.6847708230655495
Test Recall: 0.7294
[[6600 3690]
 [2706 7294]]
